In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
#store the filepath of tables into variable
tablepath = "dbfs:/FileStore/tables/"

#list source files present in the tables directory
dbutils.fs.ls(tablepath)

Out[104]: [FileInfo(path='dbfs:/FileStore/tables/Technology_Skills.txt', name='Technology_Skills.txt', size=2513724, modificationTime=1684271183000),
 FileInfo(path='dbfs:/FileStore/tables/skill2vec_10K.csv', name='skill2vec_10K.csv', size=10731364, modificationTime=1684270161000),
 FileInfo(path='dbfs:/FileStore/tables/skill2vec_1K.csv', name='skill2vec_1K.csv', size=1074156, modificationTime=1684270127000),
 FileInfo(path='dbfs:/FileStore/tables/skill2vec_50K.csv.gz', name='skill2vec_50K.csv.gz', size=3221283, modificationTime=1684269611000)]

In [0]:
#filenames of Skill2vec Datasets
skill2vecName = "skill2vec_50K.csv.gz"

#skill2vecName = "skill2vec_10K.csv"
#skill2vecName = "skill2vec_1K.csv"

techskillName = "Technology_Skills.txt"

#read dataset files and store in dataframes
skill2vec = spark.read.format("csv").option("header", "false")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(tablepath + skill2vecName)
techskill = spark.read.option("header","true").option("sep", "\t")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .csv(tablepath+techskillName)

In [0]:
#count the number of rows in both the datasets
def count_it(tbl, lbl):
    print('{0:15s}: {1:,} rows'.format(lbl, tbl.count()))

count_it(skill2vec, skill2vecName)
count_it(techskill, techskillName)

skill2vec_50K.csv.gz: 50,000 rows
Technology_Skills.txt: 31,461 rows


In [0]:
#import 'col' , 'array', and 'expr' functions from pyspark sql functions
from pyspark.sql.functions import col, array, expr

#data preprocessing of skill2vec dataset
skill2vec = skill2vec.withColumnRenamed("_c0","id")
skill2vec = skill2vec.withColumn("skills", array([col for col in skill2vec.columns])).select("id", "skills")
skill2vec = skill2vec.withColumn("skills", expr("slice(skills, 2, SIZE(skills))"))

In [0]:
#import 'explode' function from pyspark sql functions
from pyspark.sql.functions import explode

#explode skill2vec dataset
df = skill2vec.select(skill2vec.id, explode(skill2vec.skills)).withColumnRenamed("col","skills")
df.show(10, truncate=False)

+------+---------------------+
|id    |skills               |
+------+---------------------+
|125720|HR Executive         |
|125720|screening            |
|125720|selection            |
|125720|Interview            |
|125720|HR                   |
|125720|Recruiter            |
|125720|IT Recruiter         |
|125720|Sourcing             |
|125720|recruitment executive|
|125720|onboarding           |
+------+---------------------+
only showing top 10 rows



In [0]:
print("Number of rows in Skill2vec dataset BEFORE removing NA values: "+ str(df.count()))
#drop NA values for df
df = df.dropna()
print("Number of rows in Skill2vec dataset AFTER removing NA values: " + str(df.count()) + "\n")

print("Number of rows in TechSkill dataset BEFORE removing NA values: "+ str(techskill.count()))
#drop NA values for TechSkill
techskill = techskill.dropna()
print("Number of rows in TechSkill dataset AFTER removing NA values: " + str(techskill.count()))

Number of rows in Skill2vec dataset BEFORE removing NA values: 48000000
Number of rows in Skill2vec dataset AFTER removing NA values: 463908

Number of rows in TechSkill dataset BEFORE removing NA values: 31461
Number of rows in TechSkill dataset AFTER removing NA values: 31461


In [0]:
print("Number of rows in Skill2vec dataset BEFORE removing duplicate values: " + str(df.count()))
#drop duplicate values for df
df = df.drop_duplicates()
print("Number of rows in Skill2vec dataset AFTER removing duplicate values: " + str(df.count()) + "\n")

print("Number of rows in TechSkill dataset BEFORE removing duplicate values: " + str(techskill.count()))
#drop duplicate values for TechSkill
techskill = techskill.drop_duplicates()
print("Number of rows in TechSkill dataset AFTER removing duplicate values: " + str(techskill.count()))

Number of rows in Skill2vec dataset BEFORE removing duplicate values: 463908
Number of rows in Skill2vec dataset AFTER removing duplicate values: 463803

Number of rows in TechSkill dataset BEFORE removing duplicate values: 31461
Number of rows in TechSkill dataset AFTER removing duplicate values: 31461


In [0]:
skill2vec.printSchema()
techskill.printSchema()

root
 |-- id: string (nullable = true)
 |-- skills: array (nullable = false)
 |    |-- element: string (containsNull = true)

root
 |-- O*NET-SOC Code: string (nullable = true)
 |-- Example: string (nullable = true)
 |-- Commodity Code: string (nullable = true)
 |-- Commodity Title: string (nullable = true)
 |-- Hot Technology: string (nullable = true)
 |-- In Demand: string (nullable = true)



In [0]:
#Question 1

#print distinct job descriptions
print("Distinct Job Descriptions: " + str(df.select("id").distinct().count()))

Distinct Job Descriptions: 50000


In [0]:
#Question 1 -  Further
print("Distinct Skills: " + str(df.select("skills").distinct().count()) + "\n")

#data insights
print("Maximum Number of Skills: ")
df.groupBy("id").count().sort("count", ascending=False).show(1)

print("Minimum Number of Skills: ")
df.groupBy("id").count().sort("count", ascending=True).show(1)

Distinct Skills: 70579

Maximum Number of Skills: 
+----+-----+
|  id|count|
+----+-----+
|9496|  960|
+----+-----+
only showing top 1 row

Minimum Number of Skills: 
+------+-----+
|    id|count|
+------+-----+
|111463|    1|
+------+-----+
only showing top 1 row



In [0]:
#Question 2

df2 = df.groupBy("skills")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "frequency")

#print results
df2.show(10, truncate=False)

+--------------------+---------+
|skills              |frequency|
+--------------------+---------+
|Java                |1911     |
|Javascript          |1770     |
|Sales               |1705     |
|Business Development|1545     |
|Web Technologies    |1313     |
|Communication Skills|1305     |
|development         |1238     |
|Marketing           |1184     |
|Finance             |1078     |
|HTML                |1067     |
+--------------------+---------+
only showing top 10 rows



In [0]:
#Question 3

#import 'countDistinct' function from pyspark sql functions
from pyspark.sql.functions import countDistinct

df3 = df.groupBy("id")\
        .agg(countDistinct("skills"))

df3 = df3.groupBy("count(skills)")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "Frequency")\
        .withColumnRenamed("count(skills)", "Skill Count")

#print results
df3.show(5,truncate=False)

+-----------+---------+
|Skill Count|Frequency|
+-----------+---------+
|10         |10477    |
|5          |3432     |
|6          |3405     |
|1          |3386     |
|7          |3345     |
+-----------+---------+
only showing top 5 rows



In [0]:
#Question 3 - Further
#display skills which contain only special characters
df.filter(col("skills").rlike("^(?:[^a-zA-Z0-9\s]+)$"))\
        .groupBy("skills")\
        .count()\
        .withColumnRenamed("skills", "Non-Alphanumeric Skills")\
        .show()

#remove skills which are not alphanumeric
df3_1 = df.filter(~col("skills").rlike("^(?:[^a-zA-Z0-9\s]+)$"))

df3_1 = df3_1.groupBy("id")\
        .agg(countDistinct("skills"))

df3_1 = df3_1.groupBy("count(skills)")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "Frequency")\
        .withColumnRenamed("count(skills)", "Skill Count")

#print results
df3_1.show(5,truncate=False)

+-----------------------+-----+
|Non-Alphanumeric Skills|count|
+-----------------------+-----+
|                    ...|   14|
|                     ++|    1|
|                      .|    7|
|           óùô_ô¬¾ïù|    1|
|              ¾ñ´¾ïâ»_|    2|
+-----------------------+-----+

+-----------+---------+
|Skill Count|Frequency|
+-----------+---------+
|10         |10477    |
|5          |3429     |
|6          |3406     |
|1          |3386     |
|7          |3345     |
+-----------+---------+
only showing top 5 rows



In [0]:
#Question 4

#import 'lower' function from pyspark sql functions
from pyspark.sql.functions import lower

#create dataset with skills listed in lowercase
df_l = df.withColumn("skills", lower(col("skills")))

df4 = df_l.groupBy("skills")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("skills", "Skill")\
        .withColumnRenamed("count", "Frequency")

#print results
df4.show(10,truncate=False)

+--------------------+---------+
|Skill               |Frequency|
+--------------------+---------+
|java                |2759     |
|javascript          |2738     |
|sales               |2680     |
|business development|2108     |
|marketing           |1809     |
|sql                 |1564     |
|jquery              |1547     |
|html                |1539     |
|communication skills|1537     |
|bpo                 |1530     |
+--------------------+---------+
only showing top 10 rows



In [0]:
#Question 4 - Further
print("Distinct Skills: " + str(df_l.select("skills").distinct().count()) + "\n")

#import 'regexp_replace' function from pyspark sql functions
from pyspark.sql.functions import regexp_replace

#data insights
df_l.filter(col("skills").rlike("^java\w+script$"))\
        .groupBy("skills")\
        .count()\
        .show()

#replace java_script with javascript
df4_1 = df_l.select("id", regexp_replace("skills", "^java\w+script$", "javascript").alias('skills'))

df4_1 = df4_1.groupBy("skills")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("skills", "Skill")\
        .withColumnRenamed("count", "Frequency")

#print results
df4_1.show(5,truncate=False)

Distinct Skills: 55545

+-----------+-----+
|     skills|count|
+-----------+-----+
|java_script|   28|
+-----------+-----+

+--------------------+---------+
|Skill               |Frequency|
+--------------------+---------+
|javascript          |2766     |
|java                |2759     |
|sales               |2680     |
|business development|2108     |
|marketing           |1809     |
+--------------------+---------+
only showing top 5 rows



In [0]:
#Question 5

#create dataset with example column of techskill dataset listed in lowercase
tech_l = techskill.withColumn("Example", lower(col("Example")))

#join
df5 = df_l.join(tech_l, df_l.skills ==  tech_l.Example, "inner")

#print results
print("Skills Before Join: " + str(df_l.count()))
print("Skills After Join: " + str(df5.count()))

Skills Before Join: 463803
Skills After Join: 1101498


In [0]:
#Question 5 further
print("MS in Skill2Vec Dataset:")
df_l.filter(col("skills").rlike("(?i)^ms "))\
        .groupBy("skills")\
        .count()\
        .sort('count', ascending=False)\
        .show(5)

print("Microsoft in Skill2Vec Dataset:" + 
      str(df_l.filter(col("skills").rlike("(?i)^microsoft ")).count()) + 
      "\n")

print("MS in O*Net Dataset:" + 
      str(tech_l.filter(col("Example").rlike("(?i)^ms ")).count()) + 
      "\n")

print("Microsoft in O*Net Dataset:")
tech_l.filter(col("Example").rlike("(?i)^microsoft "))\
        .groupBy("Example")\
        .count()\
        .sort('count', ascending=False)\
        .show(5)

#replace ms with microsoft
df_l5 = df_l.select("id", regexp_replace("skills", "(?i)^ms ", "microsoft ").alias('skills'))

#join
df5_1 = df_l5.join(tech_l, df_l5.skills ==  tech_l.Example, "inner")

#print results
print("Skills Before Join: " + str(df_l.count()))
print("Skills After Join: " + str(df5_1.count()))

MS in Skill2Vec Dataset:
+-------------+-----+
|       skills|count|
+-------------+-----+
|    ms office|  454|
|ms sql server|  167|
|       ms sql|  125|
|     ms excel|   80|
|    ms access|   71|
+-------------+-----+
only showing top 5 rows

Microsoft in Skill2Vec Dataset:335

MS in O*Net Dataset:0

Microsoft in O*Net Dataset:
+--------------------+-----+
|             Example|count|
+--------------------+-----+
|     microsoft excel|  844|
|microsoft office ...|  778|
|      microsoft word|  773|
|microsoft powerpoint|  597|
|   microsoft outlook|  535|
+--------------------+-----+
only showing top 5 rows

Skills Before Join: 463803
Skills After Join: 1259025


In [0]:
#Question 6

df6 = df5.groupBy("Commodity Title")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "Frequency")

#print results
df6.show(10,truncate=False)

+-------------------------------------------------+---------+
|Commodity Title                                  |Frequency|
+-------------------------------------------------+---------+
|Object or component oriented development software|324521   |
|Web platform development software                |298754   |
|Operating system software                        |190926   |
|Development environment software                 |53013    |
|Data base management system software             |44132    |
|Analytical or scientific software                |33552    |
|Web page creation and editing software           |31682    |
|Data base user interface and query software      |29436    |
|Spreadsheet software                             |18568    |
|File versioning software                         |13846    |
+-------------------------------------------------+---------+
only showing top 10 rows



In [0]:
#Question 6 - Further
print("Commodity Titles in Demand:")
df6_1 = df5.filter(col("In Demand") == 'Y')\
        .groupBy("Commodity Title")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "Frequency")
df6_1.show(10, truncate=False)

print("Commodity Titles for Hot Technologies:")
df6_2 = df5.filter(col("Hot Technology") == 'Y')\
        .groupBy("Commodity Title")\
        .count()\
        .sort('count', ascending=False)\
        .withColumnRenamed("count", "Frequency")
df6_2.show(10, truncate=False)

Commodity Titles in Demand:
+-------------------------------------------------+---------+
|Commodity Title                                  |Frequency|
+-------------------------------------------------+---------+
|Object or component oriented development software|95370    |
|Web platform development software                |51228    |
|Operating system software                        |35567    |
|Development environment software                 |11308    |
|Spreadsheet software                             |8294     |
|File versioning software                         |4186     |
|Analytical or scientific software                |3422     |
|Data base management system software             |1976     |
|Web page creation and editing software           |1462     |
|Word processing software                         |1386     |
+-------------------------------------------------+---------+
only showing top 10 rows

Commodity Titles for Hot Technologies:
+--------------------------------------